In [138]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
pd.set_option('max_colwidth',1000)
pd.set_option('display.width',1000)

import string
import random

import numpy as np
np.set_printoptions(suppress=True)

import matplotlib.pyplot as plt
plt.style.use('seaborn-darkgrid')

import warnings
warnings.filterwarnings('ignore')


from sklearn.metrics import (classification_report,
                             confusion_matrix 
                            )


from yellowbrick.classifier import ConfusionMatrix, ROCAUC, PrecisionRecallCurve
 
import spacy

# Classificando textos com NLP

In [3]:
# versão do algoritmo de nlp
spacy.__version__

'2.3.5'

In [4]:
!python -m spacy download pt

[+] Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')
[x] Couldn't link model to 'pt'
Creating a symlink in spacy/data failed. Make sure you have the required
permissions and try re-running the command as admin, or use a virtualenv. You
can still import the model as a module and call its load() method, or create the
symlink manually.
C:\Users\erico\anaconda3\envs\POC_eric\lib\site-packages\pt_core_news_sm -->
C:\Users\erico\anaconda3\envs\POC_eric\lib\site-packages\spacy\data\pt
[!] Download successful but linking failed
Creating a shortcut link for 'pt' didn't work (maybe you don't have admin
permissions?), but you can still load the model via its full package name: nlp =
spacy.load('pt_core_news_sm')


Vocˆ nÆo tem privil‚gios suficientes para realizar esta opera‡Æo.


In [47]:
df_treino = pd.read_excel('Base_de_dados_sintética .xlsx')
df_treino.head()

,Texto do lead,Possivel Necessidade
0,Os clientes estão deixando o meu negócio,DS
1,Os clientes estão deixando o meu negócio,DS
2,Os clientes estão deixando o meu negócio,DS
3,Os clientes estão deixando o meu negócio,DS
4,Os clientes estão deixando o meu negócio,DS


In [52]:
df_teste = pd.read_csv('teste_texto.txt', encoding='UTF-8')
df_teste.head()

,Texto do lead,Possivel Necessidade
0,estamos com problemas de stock out,DS
1,não sei como obter mais clientes,DS
2,queria saber quanto iremos vender no próximo mês,DS
3,qual produto colocamos na prateleira,DS
4,qual peça de roupa vai vender mais,DS


In [62]:
df_treino.shape

(2099, 2)

In [63]:
df_treino.columns

Index(['Texto do lead', 'Possivel Necessidade'], dtype='object')

In [64]:
df_text['necessidade'].value_counts()

DS    36
BI    26
DE    24
SI    14
Name: necessidade, dtype: int64

In [65]:
# padrão de elementos de caracteres
pontuacoes = string.punctuation
pontuacoes

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [66]:
# stop words
from spacy.lang.pt.stop_words import STOP_WORDS
stop_words = STOP_WORDS

In [67]:
print(stop_words)

{'tentaram', 'quinze', 'aqui', 'todas', 'deste', 'pouca', 'perto', 'conhecida', 'cento', 'nossos', 'menos', 'vossa', 'dessa', 'devem', 'estar', 'são', 'desta', 'sempre', 'tendes', 'dão', 'à', 'meu', 'sua', 'neste', 'vais', 'iniciar', 'pela', 'fazemos', 'pelos', 'quinta', 'foram', 'veja', 'longe', 'nova', 'cedo', 'dezasseis', 'podia', 'alguns', 'nossa', 'partir', 'qual', 'onze', 'embora', 'vai', 'des', 'estás', 'lado', 'teu', 'catorze', 'ora', 'dez', 'quinto', 'ali', 'irá', 'pode', 'contra', 'máximo', 'final', 'vossos', 'demais', 'tais', 'corrente', 'posição', 'vezes', 'for', 'forma', 'parte', 'mil', 'oitavo', 'sexta', 'vosso', 'é', 'às', 'sete', 'era', 'último', 'estiveram', 'és', 'seu', 'ambos', 'aquelas', 'nas', 'fui', 'do', 'dezassete', 'mesmo', 'sétima', 'isto', 'segundo', 'sem', 'pôde', 'muitos', 'daquele', 'dá', 'fará', 'oito', 'relação', 'breve', 'dentro', 'quieto', 'este', 'de', 'ontem', 'vem', 'através', 'todo', 'duas', 'ou', 'povo', 'quer', 'um', 'pegar', 'querem', 'doze', 't

In [68]:
len(stop_words)

413

In [69]:
# carrega dicionario pt br
pln = spacy.load("pt_core_news_sm")

In [70]:
pln

In [71]:
def preprocess(texto):
    '''
    Função para retirar as stop_word e números do texto
    '''
    texto = texto.lower()
    documento = pln(texto)
    
    lista = []
    for token in documento:
        
        lista.append(token.lemma_)
    lista = [palavra for palavra in lista if palavra not in stop_words and palavra not in pontuacoes]  
    lista = ' '.join( [str(elemento) for elemento in lista if not elemento.isdigit()] )
    
    
    return lista
    

In [72]:
teste = preprocess('Estou tentando Resolver problemas e facilitar as decisões Das Pessoas 15 11 2021')
teste

'resolver problema e facilitar o decisão pessoa'

In [141]:
teste2 = preprocess('temos a ferramenta de BI')
teste2

'o ferramenta bi'

- Preprocessamento da base de treino

In [77]:
df_treino.head()

,Texto do lead,Possivel Necessidade
0,Os clientes estão deixando o meu negócio,DS
1,Os clientes estão deixando o meu negócio,DS
2,Os clientes estão deixando o meu negócio,DS
3,Os clientes estão deixando o meu negócio,DS
4,Os clientes estão deixando o meu negócio,DS


In [20]:
df_text['texto'] = df_text['texto'].apply(preprocess)

In [78]:
# exemplo de base de entrada
exemplo_base_dados = [
    ["estamos com problemas de stock out", {"DS":True, "BI":False, "DE":False, "SI":False}],
    ["o banco de dados da empresa é lento",{"DS":False, "BI":False, "DE":True, "SI":False}],
    ["não temos uma ferramenta de BI na empresa",{"DS":False, "BI":True, "DE":False, "SI":False}],
    ["queremos um novo layout no site",{"DS":False, "BI":False, "DE":False, "SI":True}]
]

In [ ]:
type(exemplo_base_dados)

In [81]:
exemplo_base_dados[0]

['estamos com problemas de stock out',
 {'DS': True, 'BI': False, 'DE': False, 'SI': False}]

In [82]:
exemplo_base_dados[0][0]

'estamos com problemas de stock out'

In [83]:
exemplo_base_dados[0][1]

{'DS': True, 'BI': False, 'DE': False, 'SI': False}

In [84]:
type(exemplo_base_dados[0][1])

dict

In [88]:
# alterar para o padrão de dict as entradas
base_dados_final = []

for texto, necessidade in zip(df_treino['Texto do lead'], df_treino['Possivel Necessidade']):
    #print(texto, necessidade)
    
    if necessidade == 'DS':
        dic = ({"DS":True, "BI":False, "DE":False, "SI":False})
        
    elif necessidade == 'BI':
        dic = ({"DS":False, "BI":True, "DE":False, "SI":False})
        
    elif necessidade == 'DE':
        dic = ({"DS":False, "BI":False, "DE":True, "SI":False})
        
    elif necessidade == 'SI':
        dic = ({"DS":False, "BI":False, "DE":False, "SI":True})
        
    else:
        print('deu erro')
        
    base_dados_final.append([texto, dic.copy()])

In [89]:
len(base_dados_final)

2099

In [90]:
base_dados_final[0]

['Os clientes estão deixando o meu negócio ',
 {'DS': True, 'BI': False, 'DE': False, 'SI': False}]

In [91]:
base_dados_final[0][0]

'Os clientes estão deixando o meu negócio '

In [92]:
base_dados_final[0][1]

{'DS': True, 'BI': False, 'DE': False, 'SI': False}

In [93]:
type(base_dados_final[0][1])

dict

In [94]:
base_dados_final

[['Os clientes estão deixando o meu negócio ',
  {'DS': True, 'BI': False, 'DE': False, 'SI': False}],
 ['Os clientes estão deixando o meu negócio ',
  {'DS': True, 'BI': False, 'DE': False, 'SI': False}],
 ['Os clientes estão deixando o meu negócio ',
  {'DS': True, 'BI': False, 'DE': False, 'SI': False}],
 ['Os clientes estão deixando o meu negócio ',
  {'DS': True, 'BI': False, 'DE': False, 'SI': False}],
 ['Os clientes estão deixando o meu negócio ',
  {'DS': True, 'BI': False, 'DE': False, 'SI': False}],
 ['Os clientes estão deixando o meu negócio ',
  {'DS': True, 'BI': False, 'DE': False, 'SI': False}],
 ['Os clientes estão deixando o meu negócio ',
  {'DS': True, 'BI': False, 'DE': False, 'SI': False}],
 ['Os clientes estão deixando o meu negócio ',
  {'DS': True, 'BI': False, 'DE': False, 'SI': False}],
 ['Os clientes estão deixando o meu negócio ',
  {'DS': True, 'BI': False, 'DE': False, 'SI': False}],
 ['Os clientes estão deixando o meu negócio ',
  {'DS': True, 'BI': False

# criar classificador

In [95]:
# cria a instacia do modelo
modelo = spacy.blank('pt')
categorias = modelo.create_pipe('textcat')
categorias.add_label('DS')
categorias.add_label('BI')
categorias.add_label('DE')
categorias.add_label('SI')
modelo.add_pipe(categorias)

historico = [] 

In [96]:
modelo

In [97]:
categorias

In [98]:
modelo.pipe_names

['textcat']

In [100]:
from tqdm import tqdm
# classe de treino
modelo.begin_training()

for epoca in tqdm(range(100)):
    random.shuffle(base_dados_final)
    losses = {}
    for batch in spacy.util.minibatch(base_dados_final, 20):
        textos = [modelo(texto) for texto, entities in batch]
        annotations = [{'cats':entities} for texto, entities in batch]
        modelo.update(textos, annotations, losses=losses)
    if epoca % 100 == 0:
        print(losses)
        historico.append(losses)

  1%|▊                                                                                 | 1/100 [00:12<20:49, 12.62s/it]

{'textcat': 5.274551161432717e-05}


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [29:05<00:00, 17.46s/it]


In [111]:
historico_loss = []

for i in historico:
    print(i)
    historico_loss.append(i.get('textcat'))
    

{'textcat': 0.02007914623209217}
{'textcat': 5.274551161432717e-05}


In [112]:
# facilitar gráfico
historico_loss

[0.02007914623209217, 5.274551161432717e-05]

# Carga

In [113]:
# salva o modelo em dico
modelo.to_disk('modelo_nlp')

In [114]:
# carrega o modelo salvo
modelo_nlp = spacy.load('modelo_nlp')
modelo_nlp

In [115]:
texto_teste = 'nossas taxas de conversão estão abaixo do consideravél'

In [116]:
texto_teste = preprocess(texto_teste)
texto_teste

'taxar conversão abaixar consideravél'

In [117]:
previsao = modelo_nlp(texto_teste)
previsao

taxar conversão abaixar consideravél

In [118]:
previsao.cats

{'DS': 0.004916884936392307,
 'BI': 0.9999545812606812,
 'DE': 0.01463088858872652,
 'SI': 0.0014877192443236709}

In [119]:
texto_teste2 = 'precisamos mensurar a qualidade do nosso serviço'
previsao = modelo_nlp(preprocess(texto_teste2))
previsao.cats

{'DS': 0.001297717564739287,
 'BI': 0.9999545812606812,
 'DE': 0.0002125210885424167,
 'SI': 4.539787187241018e-05}

In [120]:
df_proba_nlp = pd.DataFrame(previsao.cats, index=[0])
df_proba_nlp

,DS,BI,DE,SI
0,0.001,1.000,0.000,0.000


# Avaliação do modelo na base de teste

In [124]:
# preprocessar a base antes do teste e avaliação
df_teste['Texto do lead'] = df_teste['Texto do lead'].apply(preprocess)

In [126]:
df_teste.tail()

,Texto do lead,Possivel Necessidade
95,conseguir obter informação hábil,DE
96,inteligência negócio,BI
97,kpi e métrico organizar,BI
98,site taxar conversão,SI
99,taxar conversão abaixar consideravél,DS


In [127]:
# prever cada dado no teste
previsoes = []
for texto in df_teste['Texto do lead']:
    #print(texto)
    previsao = modelo_nlp(texto)
    previsoes.append(previsao.cats)
    

In [129]:
# Preprocessar a base para adicionar as chaves no formato da base de treino
previsoes_final = []
for previsao in previsoes:
    if previsao['DS'] > previsao['BI'] and previsao['DE'] and previsao['SI']:
        previsoes_final.append('DS')
        
    elif previsao['BI'] > previsao['DS'] and previsao['DE'] and previsao['SI']:
        previsoes_final.append('BI')
        
    elif previsao['DE'] > previsao['DS'] and previsao['BI'] and previsao['SI']:
        previsoes_final.append('DE')
        
    elif previsao['SI'] > previsao['DS'] and previsao['BI'] and previsao['DE']:
        previsoes_final.append('SI')
        
    else :
        print('erro')
        
previsoes_final = np.array(previsoes_final)

In [130]:
# pegar as respostas reais para comparar com as previsões
resposta_reais = df_teste['Possivel Necessidade'].values

In [134]:
# avaliar
print(classification_report(resposta_reais, previsoes_final))

              precision    recall  f1-score   support

          BI       0.27      0.65      0.38        26
          DE       0.75      0.25      0.38        24
          DS       0.46      0.36      0.41        36
          SI       0.00      0.00      0.00        14

    accuracy                           0.36       100
   macro avg       0.37      0.32      0.29       100
weighted avg       0.42      0.36      0.33       100



In [140]:
print(confusion_matrix(resposta_reais, previsoes_final))

[[17  2  7  0]
 [14  6  4  0]
 [23  0 13  0]
 [10  0  4  0]]
